<center><img src="../图片数据/logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2019, Shenzhen Yahboom Tech  
@Author: Malloy.Yuan  
@Date: 2019-07-17 10:10:02  
@LastEditors: Malloy.Yuan  
@LastEditTime: 2019-09-17 17:54:19  

### 导入相关驱动库

In [1]:
import time
import Adafruit_SSD1306
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
from jetbot.utils.utils import get_ip_address
import subprocess

### 初始化SSD1306

In [2]:
# 128x32 显示与硬件I2C:
disp = Adafruit_SSD1306.SSD1306_128_32(rst=None, i2c_bus=0, gpio=1)# 将gpio设置hack为1，以避免平台检测
# 初始化库。
disp.begin()
# 清除显示
disp.clear()
disp.display()
# 为绘图创建空白图像
# 确保创建带有模式为'1'即1位颜色的图像
width = disp.width
height = disp.height
image = Image.new('1', (width, height))
# 获取要在图像上绘制的绘图对象
draw = ImageDraw.Draw(image)
# 画一个黑色填充框来清除图像
draw.rectangle((0,0,width,height), outline=0, fill=0)
# 画一些形状
# 首先定义一些常量，以方便调整形状的大小
padding = -2
top = padding
bottom = height-padding
# 从左到右移动，以跟踪绘图图形的当前x位置。
x = 0
# 加载默认字体
font = ImageFont.load_default()

### 每隔一秒循环刷新信息显示到OLED屏

In [3]:
while True:

    # 画一个黑色填充框来清除图像。
    draw.rectangle((0,0,width,height), outline=0, fill=0)

    # 从这个链接可以获取系统监视的Shell脚本 : 
    #https://unix.stackexchange.com/questions/119126/command-to-display-memory-usage-disk-usage-and-cpu-load
    cmd = "top -bn1 | grep load | awk '{printf \"CPU Load: %.2f\", $(NF-2)}'"
    CPU = subprocess.check_output(cmd, shell = True )
    cmd = "free -m | awk 'NR==2{printf \"Mem:%s/%sM %.2f%%\", $3,$2,$3*100/$2 }'"
    MemUsage = subprocess.check_output(cmd, shell = True )
    cmd = "df -h | awk '$NF==\"/\"{printf \"Disk:%d/%dGB %s\", $3,$2,$5}'"
    Disk = subprocess.check_output(cmd, shell = True )

    draw.text((x, top),       "eth0:" + str(get_ip_address('eth0')),  font=font, fill=255)
    draw.text((x, top+8),     "wlan0:" + str(get_ip_address('wlan0')), font=font, fill=255)
    draw.text((x, top+16),    str(MemUsage.decode('utf-8')),  font=font, fill=255)
    draw.text((x, top+25),    str(Disk.decode('utf-8')),  font=font, fill=255)

    # 显示图像
    disp.image(image)
    disp.display()
    time.sleep(1)

KeyboardInterrupt: 